In [95]:
import pandas as pd
import json
import os
from collections import defaultdict
import configparser

In [106]:
torch_results_dir = os.path.join(os.getcwd(), os.pardir, "saved_models", "torch")
torch_results_files = [os.path.join(torch_results_dir, f, "results.json") for f in os.listdir(torch_results_dir)]

keras_results_dir = os.path.join(os.getcwd(), os.pardir, "saved_models", "keras")
keras_results_files = [os.path.join(keras_results_dir, f, "results.json") for f in os.listdir(keras_results_dir)]

In [105]:
# torch_files = []

# for f in torch_results_files:
#     if "scibert" in f:
#         sub_folder = f.split("results.json")[0]
#         sub_files = os.listdir(sub_folder)
#         print(sub_folder)
#         sub_files = [os.path.join(f, sf, "results.json") for sf in sub_files]
#         for sf in sub_files:
#             torch_files.append(sf)
#     else:
#         torch_files.append(f)

# torch_results_files = torch_files

In [109]:
def result_to_df(files, _type):
    
    type_mapping = {"torch": ["model_version", ["lemmatize", "balance_dataset", "shuffle_data", "epochs", "batch_size", "learning_rate", "max_len", "bfloat16"]], 
                "keras": ["embedding_type", ["lemmatize", "balance_dataset", "number_of_epochs", "batch_size", "learning_rate", "max_len"]]}
    model_type, params = type_mapping[_type]
    
    results = []
    for file in files:

        if not os.path.isfile(file):
            continue
        with open(file, "r") as f:
            res = json.load(f)
            res["model"] = [x for x in file.split("&") if model_type in x][0].split("=")[-1]

            config = configparser.ConfigParser()
            config.read(file.replace("results.json", "config.txt"))

            for param in params:
                res[param] = config["param"][param]

            results.append(res)
            # break
    
    headers = ["precision", "recall", "f1-score", "0_precision", "0_recall", "0_f1-score", "1_precision", "1_recall", "1_f1-score", "2_precision", "2_recall", "2_f1-score"]
    mapping = {"precision": "p", "recall": "r", "f1-score": "f1", "0-precision": "0-p", "0-recall": "0-r", "0-f1-score": "0-f1", "1-precision": "1-p", "1-recall": "1-r", "1-f1-score": "1-f1", "2-precision": "2-p", "2-recall": "2-r", "2-f1-score": "2-f1", "model": "model", "lemmatize": "l", "balance_dataset": "b", "shuffle_data": "s", "epochs": "epoch", "number_of_epochs": "epoch", "batch_size": "batch", "learning_rate": "lr", "max_len": "len", "bfloat16": "bf16"}
    
    result_table = defaultdict(list)

    for result in results:
        result_table[mapping["model"]].append(result["model"])
        result_table[mapping["lemmatize"]].append(result["lemmatize"][0])
        result_table[mapping["balance_dataset"]].append(result["balance_dataset"][0])
        if _type == "torch":
            result_table[mapping["shuffle_data"]].append(result["shuffle_data"][0])
            result_table[mapping["bfloat16"]].append(result["bfloat16"][0])
            result_table[mapping["epochs"]].append(result["epochs"])
        elif _type == "keras":
            result_table[mapping["number_of_epochs"]].append(result["number_of_epochs"])
        result_table[mapping["batch_size"]].append(result["batch_size"])
        result_table[mapping["learning_rate"]].append(result["learning_rate"])
        result_table[mapping["max_len"]].append(result["max_len"])
        for header in headers:
            if "_" in header:
                _class,_metric = header.split("_")
                if _type == "keras":
#                     _class = str(int(_class)+2)
                    score = result[str(int(_class)+2)][_metric]
                else:
                    score = result[_class][_metric]
                result_table[mapping[_class+"-"+_metric]].append(round(score, 3))
            else:
                _metric = header
                score = result["macro avg"][_metric]
                result_table[mapping[_metric]].append(round(score, 3))
                
    return pd.DataFrame(result_table)

In [113]:
torch_df[["model", "f1", "0-p", "0-r", "0-f1", "1-p", "1-r", "1-f1", "2-p", "2-r", "2-f1"]]

,model,f1,0-p,0-r,0-f1,1-p,1-r,1-f1,2-p,2-r,2-f1
0,albert-base-v2,0.810,0.837,0.787,0.811,0.853,0.845,0.849,0.711,0.838,0.770
1,allenai+scibert_scivocab_uncased,0.828,0.887,0.779,0.829,0.859,0.876,0.867,0.720,0.873,0.789
2,allenai+scibert_scivocab_cased,0.782,0.841,0.833,0.837,0.923,0.736,0.819,0.535,0.965,0.689
3,bert-base-cased,0.757,0.877,0.767,0.818,0.904,0.732,0.809,0.480,0.973,0.643
4,bert-base-cased,0.805,0.826,0.825,0.825,0.897,0.794,0.843,0.631,0.911,0.746
5,bert-base-uncased,0.819,0.856,0.797,0.825,0.874,0.846,0.860,0.686,0.884,0.772
6,bert-base-cased,0.843,0.865,0.815,0.839,0.887,0.858,0.872,0.733,0.923,0.817
7,albert-base-v2,0.742,0.684,0.889,0.774,0.954,0.610,0.744,0.564,0.954,0.709
8,allenai+scibert_scivocab_cased,0.821,0.824,0.858,0.840,0.913,0.787,0.845,0.660,0.946,0.778
9,bert-base-uncased,0.760,0.781,0.864,0.821,0.945,0.667,0.782,0.518,0.977,0.677


In [111]:
torch_df = result_to_df(torch_results_files, "torch")
torch_df[["model", "f1"]]

,model,f1
0,albert-base-v2,0.810
1,allenai+scibert_scivocab_uncased,0.828
2,allenai+scibert_scivocab_cased,0.782
3,bert-base-cased,0.757
4,bert-base-cased,0.805
5,bert-base-uncased,0.819
6,bert-base-cased,0.843
7,albert-base-v2,0.742
8,allenai+scibert_scivocab_cased,0.821
9,bert-base-uncased,0.760


In [61]:
keras_df = result_to_df(keras_results_files, "keras")
keras_df

,model,l,b,epoch,batch,lr,len,p,r,f1,0-p,0-r,0-f1,1-p,1-r,1-f1,2-p,2-r,2-f1
0,lstm/results.json,T,F,5,64,0.99,100,0.739,0.760,0.748,0.855,0.772,0.811,0.725,0.810,0.765,0.637,0.699,0.667
1,albert/results.json,F,F,5,64,0.99,50,0.108,0.333,0.164,0.000,0.000,0.000,0.325,1.000,0.491,0.000,0.000,0.000
2,lstm/results.json,F,F,5,64,0.99,100,0.782,0.756,0.767,0.803,0.852,0.827,0.803,0.764,0.783,0.738,0.653,0.693
3,lstm/results.json,F,F,5,64,0.99,100,0.712,0.723,0.714,0.783,0.808,0.795,0.789,0.673,0.726,0.565,0.687,0.620
4,lstm/results.json,F,F,5,64,0.99,100,0.739,0.748,0.742,0.798,0.814,0.806,0.786,0.716,0.749,0.634,0.714,0.672
5,lstm/results.json,F,F,20,64,0.99,100,0.676,0.697,0.684,0.767,0.747,0.757,0.727,0.688,0.707,0.535,0.656,0.589
6,lstm/results.json,T,F,20,64,0.99,100,0.711,0.741,0.723,0.815,0.754,0.783,0.742,0.759,0.750,0.577,0.710,0.637
7,lstm/results.json,F,F,20,64,0.99,100,0.704,0.728,0.712,0.791,0.772,0.782,0.785,0.714,0.748,0.536,0.699,0.606
8,lstm/results.json,T,F,5,64,0.99,100,0.769,0.725,0.743,0.782,0.867,0.822,0.796,0.721,0.756,0.731,0.587,0.651
9,lstm/results.json,F,F,20,64,0.99,100,0.727,0.673,0.693,0.745,0.854,0.795,0.765,0.663,0.710,0.670,0.502,0.574


In [ ]:
print(df.to_latex(columns=['model', 'p', 'r',
       'f1', '0-p', '0-r', '0-f1', '1-p', '1-r', '1-f1', '2-p', '2-r', '2-f1'], index=False))